In [ ]:
import time, random
from selenium import webdriver
from db_conn import push_prod_data, push_prod_data2

driver = webdriver.Chrome('chromedriver')
driver.implicitly_wait(2)

def scroll_down():
    last_ele = driver.find_element_by_css_selector('.list_basis > div > div:last-child')
    driver.execute_script('arguments[0].scrollIntoView()', last_ele)
    return len(driver.find_elements_by_css_selector('.list_basis > div > div'))

def scroll_ready():
    g_cnt = len(driver.find_elements_by_css_selector('.list_basis > div > div'))
    while g_cnt < 80:
        g_cnt = scroll_down()
        time.sleep(random.randint(1, 2))

def next_page():
    try:
        driver.find_element_by_css_selector('a[class^="pagination_next"]').click()
    except:
        return False
    time.sleep(random.randint(1, 2))
    return True


def get_items():
    node_list = driver.find_elements_by_css_selector('.list_basis > div > div')
    for node in node_list:
        a = node.find_element_by_css_selector('div[class^="basicList_price_area"]')
        driver.execute_script('arguments[0].scrollIntoView()', a)
        if '광고' not in a.text:
            t = node.find_element_by_css_selector('div[class^="basicList_title"] > a')
            prod_name = t.get_attribute('title')
            prod_name = prod_name.replace("'", "")
            thumb = ''
            try:
                thumb_tag = node.find_element_by_css_selector('a[class^="thumbnail_thumb"] > img')
                thumb = thumb_tag.get_attribute('src')
            except:
                print('thumb err', prod_name)

            price_temp = a.find_elements_by_css_selector('strong[class^="basicList_price"] > span > span')
            if len(price_temp) > 0:
                zzim = node.find_element_by_css_selector('button[class^="basicList_btn_zzim"]')
                parent_pcode = zzim.get_attribute('data-i')
                desc = node.find_element_by_css_selector('div[class^="basicList_depth"]').text
                desc += node.find_element_by_css_selector('div[class^="basicList_detail_box"]').text
                push_prod_data({
                    'pcode': parent_pcode,
                    'prod_name': prod_name,
                    'thumb': thumb,
                    'desc': desc,
                    'href': 'https://search.shopping.naver.com/catalog/'+parent_pcode,
                })

# 강아지
driver.get('https://search.shopping.naver.com/search/category?catId=50006630&frm=NVSHMDL&origQuery&pagingIndex=1&pagingSize=80&productSet=model&query&sort=rel&timestamp=&viewType=list')
# 고양이
# driver.get('https://search.shopping.naver.com/search/category?catId=50006678&frm=NVSHCAT&origQuery&pagingIndex=1&pagingSize=80&productSet=model&query&sort=rel&timestamp=&viewType=list')

In [ ]:
cnt = 0
if cnt == 0: scroll_ready()

flag = True
while flag:
    cnt += 1
    print(cnt)
    get_items()
    flag = next_page()

driver.quit()
